In [1]:
import os
%pwd

'/Users/omkarthakur/Desktop/Text-Summary/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/omkarthakur/Desktop/Text-Summary'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str



# Configuration Manager :

# Before that we have to import libraries form the constants to get path and necessary infop

In [5]:
from src.textsummarization.constants import *
from src.textsummarization.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

# Logging

In [6]:
import os
from src.textsummarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk



/Users/omkarthakur/Desktop/Text-Summary/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Components:

In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    #So we have to convert our seq to seq data in this format
    def convert_examples_to_features(self, example_batch):
        input_embeddings = self.tokenizer( example_batch['dialogue'], max_length=1024, truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_embeddings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)
        return{
            "input_ids": input_embeddings["input_ids"],
            "attention_mask": input_embeddings["attention_mask"],
            "labels": target_embeddings["input_ids"]
    }
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

# Running:

In [8]:
# Debug: Check dataset columns
from src.textsummarization.config.configuration import ConfigurationManager

config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()

dataset_samsum = load_dataset(data_transformation_config.data_path)
print("Dataset columns:", dataset_samsum['train'].column_names)
print("\nFirst example:")
print(dataset_samsum['train'][0])

[2025-10-12 03:46:32,400: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-10-12 03:46:32,402: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-12 03:46:32,403: INFO: common: created directory at: artifacts]
[2025-10-12 03:46:32,403: INFO: common: created directory at: artifacts/data_transformation]
Dataset columns: ['_data_files', '_fingerprint', '_format_columns', '_format_kwargs', '_format_type', '_output_all_columns', '_split']

First example:
{'_data_files': [{'filename': 'data-00000-of-00001.arrow'}], '_fingerprint': '41234e01aa3519db', '_format_columns': None, '_format_kwargs': {}, '_format_type': None, '_output_all_columns': False, '_split': 'train'}


In [9]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(data_transformation_config)
data_transformation.convert()

[2025-10-12 03:46:32,448: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-10-12 03:46:32,450: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-12 03:46:32,450: INFO: common: created directory at: artifacts]
[2025-10-12 03:46:32,451: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/omkarthakur/Desktop/Text-Summary/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 198114.14 examples/s]
